# Flask服务端部署
Flask是一个由PyThon编写的轻量级Web框架。通过Flask，我们可以将模型推理部署到服务端，以向服务器发送请求的方式执行推理。本节我们需要编写一个Flask app以实现该目标。

服务端部署与本地部署的最大差别在于执行推理的入口不同。对于本地部署，我们只需在本地环境中编写推理脚本并执行即可。而对于服务端部署，我们需要通过向服务端发送请求的方式执行推理。因此，Flask服务部署将在我们已经实现的本地部署的基础上完成服务端的架设。

## 一个简单的Flask App
一个简单的Flask App示例如下：
![](https://s3.bmp.ovh/imgs/2021/09/014cbaca87aa77b8.png)

当我们启动Flask服务并访问服务端根目录，该App将显示Hello World字样：
![](https://s3.bmp.ovh/imgs/2021/09/dface1f2b20a2620.png)

结合代码很容易理解，我们在函数app.route()中传入了参数‘/’，表示根目录。当我们访问该目录时将会执行该函数，而app.route()函数又将调用hello_world()函数，从而输出“Hello, World!"。在PyThon语法中，@语句被称为装饰器（Decorators）。装饰器是PyThon语言的一大特性，装饰器的合理使用可以让代码更加简洁和代码复用更加方便。有关装饰器的知识大家可以在课后查阅更多资料了解。

## 将PyTorch模型部署到Flask服务端
在本例中，我们将构建一个应用于模型推理的Flask App。该App负责监听特定的请求信息（指定访问路径和端口），并且在监听到推理请求后执行模型推理的操作，最后返回推理的结果。

首先，我们需要创建一个Flask App，该行为由Flask(\_\_name__)语句完成，其中 \_\_name__ 是一个适用于大多数情况的快捷方式。有了这个参数， Flask 才能知道在哪里可以找到模板和静态文件等部件。

然后，我们要为需要执行的函数（本例中为test()）定义一个装饰器，用于指定该函数的触发路径和请求方式。在本例中，我们将路径设置为”/test”，表示根目录下的test路径。请求方式我们设定为”POST”。

接下来我们需要在test()函数中定义Flask App的操作。当Flask服务端监听到对应的请求时，我们需要读取请求信息。在本例中，请求包含两种类型的数据，分别是测试的手写图像（字节流）以及图像的尺寸（JSON）。首先，我们通过flask的request库获取从前端发送过来的数据。由于数据的类型不同，我们也需要使用不同的方式对数据进行处理，其中我们将字节流转换为numpy数组，并根据JSON中给定的尺寸信息将numpy数组转换为对应尺寸的tensor。最后，将tensor传入推理函数run_model()中执行推理，并对推理结果进行后处理，返回推理结果。

完成Flask App的构建后，我们需要在main函数中定义App的启动函数。

**注：由于本例在Colab环境下运行，无法直接访问本地地址，因此本例采用了flask-ngrok库对flask部署地址进行映射。**

最后，启动Flask app，完成PyTorch模型在Flask服务端的部署。

In [ ]:
!pip install flask-ngrok

In [ ]:
# Download Pretrained Model
!gdown --id 1omW_gpSpedWjo5GahxzksNW-J8MGrfdk --output convnet.pth

# Download Testing Image
!gdown --id 1A47zfadA6oXGhMvI-iCRj6CWB7rS8dWz --output digit_image.jpg

In [ ]:
from flask_ngrok import run_with_ngrok
import os
import json
import numpy as np
from flask import Flask, request
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op

In [ ]:
model = ConvNet()

PATH_TO_MODEL = "./convnet.pth"
model.load_state_dict(torch.load(PATH_TO_MODEL, map_location="cpu"))
model.eval()

ConvNet(
  (cn1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (cn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (dp1): Dropout2d(p=0.1, inplace=False)
  (dp2): Dropout2d(p=0.25, inplace=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [ ]:
def run_model(input_tensor):
    model_input = input_tensor.unsqueeze(0)
    with torch.no_grad():
        model_output = model(model_input)[0]
    model_prediction = model_output.detach().numpy().argmax()
    return model_prediction

In [ ]:
def post_process(output):
    return str(output)

In [19]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/test", methods=["POST"])
def test():
    data = request.files['data'].read() #读取post请求中的'data'数据，格式为bytearray
    md = json.load(request.files['metadata']) #读取请求中的'metadata'数据
    input_array = np.frombuffer(data, dtype=np.float32) 
    input_image_tensor = torch.from_numpy(input_array).view(md["dims"]) #将'data'从numpy数组转换为形状为md["dims"]的tensor
    output = run_model(input_image_tensor)
    final_output = post_process(output)
    return final_output

if __name__ == '__main__':
    # app.run(host='0.0.0.0', port=8890) 在本地环境下使用
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cbf4-104-196-56-36.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Sep/2021 05:49:49] "POST /test HTTP/1.1" 200 -


请注意，当上述代码正确执行，将会返回如下信息，代表Flask服务已启动：

```
* Serving Flask app "__main__" (lazy loading)
 * Environment: production
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://db9c-104-196-56-36.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [03/Sep/2021 05:05:37] "POST /test HTTP/1.1" 200 -
```
接下来我们将保持该程序持续运行，进入下一节 发送推理请求 的学习